## Import libraries 

In [151]:
import pandas as pd
import numpy as np

## Convert BonBanh data and Chotot data to same format

#### Read data from file csv

In [152]:
bonbanh = pd.read_csv('Data/BonBanh.csv')
bonbanh.head()

,Tên xe,Năm sản xuất,Tình trạng,Số km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Động cơ,Màu ngoại thất,Màu nội thất,Dẫn động,Số chỗ ngồi,Giá xe
0,Xe Peugeot 2008 Active 1.2 AT,2022,Xe đã dùng,"30,000 Km",Lắp ráp trong nước,Crossover,Số tự động,Xăng 1.2 L,Đỏ,Đen,FWD - Dẫn động cầu trước,5 chỗ,679 Triệu
1,Xe Mercedes Benz C class C300 AMG,2014,Xe đã dùng,"90,000 Km",Lắp ráp trong nước,Sedan,Số tự động,Xăng 3.0 L,Trắng,Đen,RFD - Dẫn động cầu sau,5 chỗ,499 Triệu
2,Xe Mercedes Benz E class E200 Exclusive,2022,Xe đã dùng,500 Km,Lắp ráp trong nước,Sedan,Số tự động,Xăng 2.0 L,Đen,Nâu,RFD - Dẫn động cầu sau,5 chỗ,2 Tỷ 229 Triệu
3,Xe Mercedes Benz C class C200 Avantgarde,2021,Xe đã dùng,80 Km,Lắp ráp trong nước,Sedan,Số tự động,Xăng 1.5 L,Xanh,Nâu,RFD - Dẫn động cầu sau,5 chỗ,1 Tỷ 399 Triệu
4,Xe Mercedes Benz E class E200 Exclusive,2022,Xe đã dùng,"8,888 Km",Lắp ráp trong nước,Sedan,Số tự động,Xăng 2.0 L,Trắng,Nâu,RFD - Dẫn động cầu sau,5 chỗ,1 Tỷ 788 Triệu


In [153]:
chotot = pd.read_csv('Data/Chotot.csv')
chotot.head()

,Tiêu đề,Hãng,Dòng xe,Năm sản xuất,Số km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Giá
0,bán xe Xpander chính chủ,Mitsubishi,Xpander,2019,50000.0,Đã sử dụng,Tự động,Xăng,Nước khác,NaN,NaN,460.000.000 đ
1,Lên đời bán xe Pajero Sport mua hãng còn mới,Mitsubishi,Pajero Sport,2021,60000.0,Đã sử dụng,Tự động,Dầu,Thái Lan,NaN,NaN,830.000.000 đ
2,"Ford Everest Wildtrak, Đủ màu giao ngay",Ford,Everest,2023,NaN,Mới,Bán tự động,Dầu,Thái Lan,SUV / Cross over,NaN,1.409.000.000 đ
3,Everest 4x4 2023 giảm giá tới 120tr góp từ 250tr,Ford,Everest,2023,NaN,Mới,Tự động,Dầu,Thái Lan,SUV / Cross over,NaN,1.352.000.000 đ
4,cần bán xe lux A 2.0 đời 2019 Bản prelium mới đét,Vinfast,Lux A2.0,2019,26000.0,Đã sử dụng,Tự động,Xăng,Việt Nam,NaN,NaN,605.000.000 đ


#### Change value '-' into null

In [154]:
for col in bonbanh:
    bonbanh[col] = bonbanh[col].replace('-', np.nan)

#### Remove all price-missing car

In [155]:
chotot = chotot.dropna(subset=['Giá'])

#### Convert **Number of km traveled** to int and change it to *0* for all new cars

In [156]:
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].replace('Xe mới', '0 Km')
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].str.replace('Km', '')
def change_to_int(so_km):
    km = 0
    so_km = [int(x) for x in so_km.split(',')]
    n = len(so_km)
    for i in range(n):
        km += 1000**(n-i-1)*so_km[i] 
    return km
bonbanh['Số km đã đi'] = bonbanh['Số km đã đi'].apply(change_to_int)

In [157]:
chotot.loc[chotot['Tình trạng'] == 'Mới', 'Số km đã đi'] = 0

#### Convert **Price** into *Million* unit

In [158]:
def convert_price(price):
    price = price.split(" ")
    if len(price) == 4:
        return int(price[0])*1000 + int(price[2])
    else:
        if price[-1] == 'Tỷ':
            return int(price[0]) * 1000
        else:
            return int(price[0])
bonbanh['Giá xe'] = bonbanh['Giá xe'].apply(convert_price)
bonbanh = bonbanh.rename(columns = {'Giá xe': 'Giá (triệu)'})
bonbanh['Giá (triệu)'] = bonbanh['Giá (triệu)'].astype('int64')

In [159]:
chotot['Giá'] = chotot['Giá'].astype(str).str.rstrip('đ')
def price_convert(x):
    l = x.split('.')
    return ''.join(l[0:len(l)-2:1])
chotot['Giá'] = chotot['Giá'].apply(price_convert)
chotot = chotot.rename(columns = {'Giá': 'Giá (triệu)'})
chotot['Giá (triệu)'] = chotot['Giá (triệu)'].astype('int64')

#### Change **State**

In [160]:
chotot['Tình trạng'] = chotot['Tình trạng'].replace('Đã sử dụng', 'Xe đã dùng')
chotot['Tình trạng'] = chotot['Tình trạng'].replace('Mới', 'Xe mới')

#### Change **Gearbox**

In [161]:
def change_gearbox(x):   
    if x == 'Tự động' or x == '4':
        return 'Số tự động'
    elif x == 'Bán tự động':
        return 'Số hỗn hợp'
    elif x == 'Số sàn' or x == '5':
        return 'Số tay'
    else:
        return x
chotot['Hộp số'] = chotot['Hộp số'].apply(change_gearbox)

#### Change **Design** 

In [162]:
def change_design(x):
    if x == 'Bán tải / Pickup' or x == 'Pick-up (bán tải)':
        return 'Pickup'
    elif x == 'SUV' or x == 'Crossover':
        return 'SUV /Crossover'
    elif x == 'Van' or x == 'Minivan (MPV)':
        return 'Van/Minivan'
    elif x == 'Coupe (2 cửa)':
        return 'Coupe'
    elif x == 'Convertible/Cabriolet' or x == 'Mui trần':
        return 'Convertible'
    elif x == 'Truck' or x == 'Wagon':
        return 'Kiểu dáng khác'
    else:
        return x
bonbanh['Kiểu dáng'] = bonbanh['Kiểu dáng'].apply(change_design)
chotot['Kiểu dáng'] = chotot['Kiểu dáng'].apply(change_design)

#### Change **Origin** into *Domestical* or *Imported* 

In [163]:
def change_origin(x):
    if x == 'Việt Nam' or x == 'Lắp ráp trong nước':
        return 'Trong nước'
    elif x in ['Nước khác', 'Thái Lan', 'Hàn Quốc', 'Đang cập nhật', 'Mỹ', 'Nhật Bản', 'Đức', 'Đài Loan', 'Ấn Độ', 'Trung Quốc']:
        return 'Nhập khẩu'
    else:
        return x
bonbanh['Xuất xứ'] = bonbanh['Xuất xứ'].apply(change_origin)
chotot['Xuất xứ'] = chotot['Xuất xứ'].apply(change_origin)

#### Take **Capacity** and **Engine** from **Fuel type**

In [164]:
bonbanh['Dung tích'] = bonbanh['Động cơ'].str.extract(r'(\d\.\d)')
bonbanh = bonbanh.rename(columns = {'Động cơ': 'Nhiên liệu'})
bonbanh['Nhiên liệu'] = bonbanh['Nhiên liệu'].str.extract(r'(Xăng|Dầu|Hybrid|Điện)')
bonbanh.loc[bonbanh['Nhiên liệu'] == 'Điện', 'Dung tích'] = 0.0
bonbanh['Dung tích'] = bonbanh['Dung tích'].astype('float64')

#### Take **brand** from **name**

In [165]:
bonbanh = bonbanh.rename(columns = {'Tên xe': 'Hãng'})
bonbanh['Hãng'] = bonbanh['Hãng'].str.replace('Xe ', '')
check =  lambda x : ' '.join(x.split()[:1]) if x.split()[0] != "Mercedes" else ' '.join(x.split()[:2])
bonbanh['Hãng'] = bonbanh['Hãng'].apply(check)

#### Convert **Number of car seat** to int

In [166]:
bonbanh['Số chỗ ngồi'] = bonbanh['Số chỗ ngồi'].str.replace('chỗ', '')
bonbanh = bonbanh.rename(columns = {'Số chỗ ngồi': 'Số chỗ'})

#### Drop unnecessary column

In [167]:
bonbanh.drop(['Màu nội thất', 'Màu ngoại thất', 'Dẫn động', 'Dung tích'], axis=1,inplace=True)
chotot.drop(['Tiêu đề', 'Dòng xe'],axis=1,inplace=True)

#### Change position of columns

In [168]:
bonbanh = bonbanh[['Hãng', 'Năm sản xuất','Tình trạng', 'Số km đã đi', 'Xuất xứ', 'Kiểu dáng', 'Hộp số', 'Nhiên liệu', 'Số chỗ', 'Giá (triệu)']]
chotot  = chotot[['Hãng', 'Năm sản xuất','Tình trạng', 'Số km đã đi', 'Xuất xứ', 'Kiểu dáng', 'Hộp số', 'Nhiên liệu', 'Số chỗ', 'Giá (triệu)']]

In [169]:
bonbanh.head()

,Hãng,Năm sản xuất,Tình trạng,Số km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Nhiên liệu,Số chỗ,Giá (triệu)
0,Peugeot,2022,Xe đã dùng,30000,Trong nước,SUV /Crossover,Số tự động,Xăng,5,679
1,Mercedes Benz,2014,Xe đã dùng,90000,Trong nước,Sedan,Số tự động,Xăng,5,499
2,Mercedes Benz,2022,Xe đã dùng,500,Trong nước,Sedan,Số tự động,Xăng,5,2229
3,Mercedes Benz,2021,Xe đã dùng,80,Trong nước,Sedan,Số tự động,Xăng,5,1399
4,Mercedes Benz,2022,Xe đã dùng,8888,Trong nước,Sedan,Số tự động,Xăng,5,1788


In [170]:
chotot.head()

,Hãng,Năm sản xuất,Tình trạng,Số km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Nhiên liệu,Số chỗ,Giá (triệu)
0,Mitsubishi,2019,Xe đã dùng,50000.0,Nhập khẩu,NaN,Số tự động,Xăng,NaN,460
1,Mitsubishi,2021,Xe đã dùng,60000.0,Nhập khẩu,NaN,Số tự động,Dầu,NaN,830
2,Ford,2023,Xe mới,0.0,Nhập khẩu,SUV / Cross over,Số hỗn hợp,Dầu,NaN,1409
3,Ford,2023,Xe mới,0.0,Nhập khẩu,SUV / Cross over,Số tự động,Dầu,NaN,1352
4,Vinfast,2019,Xe đã dùng,26000.0,Trong nước,NaN,Số tự động,Xăng,NaN,605


## Combine **bonbanh** and **chotot**

In [171]:
data = pd.concat([bonbanh, chotot], ignore_index=True)

In [172]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38357 entries, 0 to 38356
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Hãng          38356 non-null  object 
 1   Năm sản xuất  38356 non-null  object 
 2   Tình trạng    38357 non-null  object 
 3   Số km đã đi   38094 non-null  float64
 4   Xuất xứ       34492 non-null  object 
 5   Kiểu dáng     24503 non-null  object 
 6   Hộp số        38348 non-null  object 
 7   Nhiên liệu    38349 non-null  object 
 8   Số chỗ        18889 non-null  object 
 9   Giá (triệu)   38357 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 2.9+ MB


## Convert to csv file

In [173]:
data.to_csv('Data/data.csv', index=False)